# Język Python - Wykład 4.

## Context Manager 

Problem:
    
    set things up
    do something
    tear things down

Rozwiązanie 1.

    set things up
    try:
        do something
    finally:
        tear things down

Rozwiązanie 2.

    def controlled_execution(callback):
        set things up
        try:
            callback(thing)
        finally:
            tear things down

    def my_function(thing):
        do something

    controlled_execution(my_function)

Rozwiązanie 3.

    def controlled_execution():
        set things up
        try:
            yield thing
        finally:
            tear things down

    for thing in controlled_execution():
        do something with thing

Rozwiązanie 4.

    class controlled_execution:
        def __enter__(self):
            set things up
            return thing
        def __exit__(self, type, value, traceback):
            tear things down

    with controlled_execution() as thing:
         some code

    with expression [as (targets)]:
        code block with context of "targets"

* Możliwość utworzenia dynamicznego (w runtime) kontekstu dla wykonania sekcji kodu
* Przykłady użycia: 
   * czytanie z pliku - w kontekście otwarcia i zamknięcia pliku
   * zakładanie i zwalnianie blokad ("lock")
   * zmiana i odtworzenie globalnego stanu
* Obiekt context managera musi implementować protokół: ``__enter__()`` i ``__exit__()``
* [PEP 343](http://www.python.org/dev/peps/pep-0343/)

In [1]:
with open('L4.ipynb') as notebook:
    print(notebook.read(50))

{
 "cells": [
  {
   "cell_type": "markdown",
   "


In [2]:
notebook

<_io.TextIOWrapper name='L4.ipynb' mode='r' encoding='UTF-8'>

Konstrukcja:

        with EXPR as VAR:
            BLOCK

tłumaczy się jako:

        mgr = (EXPR)
        exit = type(mgr).__exit__  # Not calling it yet
        value = type(mgr).__enter__(mgr)
        exc = True
        try:
            try:
                VAR = value  # Only if "as VAR" is present
                BLOCK
            except:
                # The exceptional case is handled here
                exc = False
                if not exit(mgr, *sys.exc_info()):
                    raise
                # The exception is swallowed if exit() returns true
        finally:
            # The normal and non-local-goto cases are handled here
            if exc:
                exit(mgr, None, None, None)

In [3]:
class MyContextManager():
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)

        
with MyContextManager() as f:
    print("Hello", f)

Context prepared
Hello None
Context closed  None None None


In [4]:
class MyContextManager():
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)
        #return True

        
with MyContextManager():
    raise Exception()

Context prepared
Context closed  <class 'Exception'>  <traceback object at 0x7f79000feb08>


Exception: 

### Context manager z użyciem dekoratora

In [5]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name,)
    yield
    print("</%s>" % name,)

with tag("h1"):
    print("foo",)

print()

with tag("div"):
    print("foo",)


<h1>
foo
</h1>

<div>
foo
</div>


Przykład z tymczasowym folderem (za http://stackoverflow.com/questions/3012488/what-is-the-python-with-statement-designed-for)

In [6]:
from tempfile import mkdtemp
from shutil import rmtree


@contextmanager
def temporary_dir(*args, **kwds):
    name = mkdtemp(*args, **kwds)
    try:
        yield name
    finally:
        rmtree(name)


with temporary_dir(".temp") as dirname:
    print("doing sth with",dirname)

doing sth with /tmp/tmpv3i4oywk.temp


## Metody specjalne c.d.

### ``__repr__`` - the “official” string representation of an object

In [7]:
class T:
    pass

print([T(), T(), T()])
print(repr(T()))
print(str(T()))

[<__main__.T object at 0x7f7900120198>, <__main__.T object at 0x7f79001201d0>, <__main__.T object at 0x7f7900120128>]


In [8]:
class T:
    def __repr__(self):
        return 'T class'

print([T(), T(), T()])
print(repr(T()))
print(str(T()))

[T class, T class, T class]
T class
T class


### ``__str__`` - the “informal” or nicely printable string representation of an object.

In [9]:
class Car:
    def __init__(self, vendor, model, ccm):
        self.vendor = vendor
        self.model = model
        self.ccm = ccm
        
    def __str__(self):
        return '%s %s (%.1f L)' % (self.vendor, self.model, self.ccm / 1000.0)

print(Car('Porsche', '911', 3600))
print(repr(Car('Porsche', '911', 3600)))
print(str(Car('Porsche', '911', 3600)))

Porsche 911 (3.6 L)
Porsche 911 (3.6 L)


W przypadku, gdy metoda __str__ nie jest zdefiniowana, wywoływana jest metoda __repr__

### ``__hash__`` - (Hash values are integers. They are used to quickly compare dictionary keys during a dictionary lookup)

In [11]:
class T:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return str(self.x)+" "+str(self.y)
        
    def __eq__(self,other):
        print("comparing to " + str(other))
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash(self.x)

d = {T(1,x) : x for x in range(100)}
T(1,10) in d

comparing to 1 1
comparing to 1 2
comparing to 1 2
comparing to 1 3
comparing to 1 3
comparing to 1 3
comparing to 1 4
comparing to 1 4
comparing to 1 4
comparing to 1 4
comparing to 1 5
comparing to 1 5
comparing to 1 5
comparing to 1 5
comparing to 1 5
comparing to 1 6
comparing to 1 6
comparing to 1 6
comparing to 1 6
comparing to 1 6
comparing to 1 6
comparing to 1 7
comparing to 1 7
comparing to 1 7
comparing to 1 7
comparing to 1 7
comparing to 1 7
comparing to 1 7
comparing to 1 7
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 8
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 9
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comparing to 1 10
comp

True

### ``__bool__`` - Called to implement truth value testing and the built-in operation

In [12]:
bool(None)

False

In [13]:
bool(0)

False

In [14]:
bool([1,3,4])

True

In [15]:
bool("")

False

In [16]:
class T:
    def __len__(self):
        return 2
print(bool(T()))

True


In [17]:
class T:
    def __bool__(self):
        print(id(self))
        if id(self) // 10 % 2 :
            return True
        else:
            return False
print(bool(T()))        

140157668844768
False


### ``__class__`` - klasa, do której należy instancja

In [18]:
x = 5
x.__class__

int

In [19]:
type(x)

int

In [20]:
type(int)

type

### Emulacja typu kontenera (cd..)

In [21]:
class T():
   
    c = [1, 2, 3]

    def __len__(self):
        return len(self.c)
    
    def __contains__(self, x):
        return x in self.c
    
    def __iter__(self):
        for x in self.c:
            yield x
    
t = T()
print(len(t))
print(2 in t)

3
True


In [22]:
iter(t)

<generator object __iter__ at 0x7f79000e6938>

In [23]:
list(iter(t))

[1, 2, 3]

## Porównanie

In [24]:
# czy jest to ta sama wartość?
a = 100000
b = 100000
a == b

True

In [25]:
# czy jest to ten sam obiekt?
a = 100000
b = 100000
a is b

False

In [26]:
# pułapki optymalizacji CPythona
a = 1
b = 1
a==b, a is b

(True, True)

In [27]:
# porównanie w przypadku klas
class T:
    pass
a = T()
b = T()
c = a
a==b,a==c

(False, True)

In [28]:
# porównanie w przypadku klas
class T:
    def __eq__(self,other):
        return True
a = T()
b = T()
a==b

True

Atrybuty

In [29]:
class Door:
    def __init__(self, number, status):
        self.number = number
        self.status = status

    def open(self):
        self.status = 'open'

    def close(self):
        self.status = 'closed'

In [30]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')
print(id(door1),id(door2))
print(id(door1.__class__),id(door2.__class__))
'0xb67e144c'

140157668845328 140157668843704
18062664 18062664


'0xb67e144c'

In [31]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    def open(self):
        self.status = 'open'

    def close(self):
        self.status = 'closed'

In [32]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')

In [33]:
print(id(Door.colour),id(door1.colour),id(door2.colour))

140157671496760 140157671496760 140157671496760


In [34]:
Door.colour, door1.colour, door2.colour

('brown', 'brown', 'brown')

In [35]:
Door.__dict__

mappingproxy({'__init__': <function Door.__init__ at 0x7f79000b8d90>, '__dict__': <attribute '__dict__' of 'Door' objects>, '__module__': '__main__', 'open': <function Door.open at 0x7f79000b8e18>, '__weakref__': <attribute '__weakref__' of 'Door' objects>, 'colour': 'brown', 'close': <function Door.close at 0x7f79000b8ea0>, '__doc__': None})

In [36]:
door1.__dict__

{'number': 1, 'status': 'closed'}

In [37]:
door1.colour = "white"

In [38]:
Door.open

<function __main__.Door.open>

In [39]:
door1.open

<bound method Door.open of <__main__.Door object at 0x7f79000b2a90>>

In [40]:
door1.__class__.__dict__['open'].__get__(door1)

<bound method Door.open of <__main__.Door object at 0x7f79000b2a90>>

## Dziedziczenie i MRO

In [41]:
class SecurityDoor(Door):
    pass

In [42]:
sdoor = SecurityDoor(1, 'closed')

In [43]:
sdoor.__dict__

{'number': 1, 'status': 'closed'}

In [44]:
SecurityDoor.__dict__

mappingproxy({'__module__': '__main__', '__doc__': None})

In [45]:
SecurityDoor.__bases__

(__main__.Door,)

In [46]:
sdoor.open

<bound method Door.open of <__main__.SecurityDoor object at 0x7f79000b5438>>

In [47]:
sdoor.__class__.__dict__['open'].__get__(sdoor)

KeyError: 'open'

In [48]:
sdoor.__class__.__bases__[0].__dict__['open'].__get__(sdoor)

<bound method Door.open of <__main__.SecurityDoor object at 0x7f79000b5438>>

In [51]:
class A:
    def foo(self):
        print("A")
    def moo(self):
        pass
class B1(A):
    def foo(self):
        print("B1")
    def moo(self):
        pass
class B2(A):
    def foo(self):
        print("B2")
    def moo(self):
        pass
class C(B1,B2):
#     def foo(self):
#         print("C")
      pass
oc = C()
oc.foo()
print(C.__mro__)

B1
(<class '__main__.C'>, <class '__main__.B1'>, <class '__main__.B2'>, <class '__main__.A'>, <class 'object'>)


### MRO

In [52]:
class X: pass
class Y: pass
class A(X,Y): pass
class B(Y,X): pass
class C(A,B): pass
C.__mro__

# https://en.wikipedia.org/wiki/C3_linearization

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

In [3]:
class F(): pass
class E(): pass
class D(): pass
class C(D,F): pass
class B(D,E): pass
class A(B,C): pass
print(A.__mro__) 

(<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.F'>, <class 'object'>)


## Wyjątki też są klasami!

In [53]:
5 + '5'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [54]:
try:
    5 + '5'
except TypeError:
    print('no-no')

no-no


In [55]:
try:
    5 + '5'
except TypeError as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [56]:
try:
    5 + '5'
except (TypeError, KeyError) as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [58]:
try:
    a[3]
except TypeError as e:
    print(type(e), e)
except KeyError as e:
    print("it's only key error")

<class 'TypeError'> 'T' object does not support indexing


In [59]:
class WrongAnswerError(Exception):
    pass

try:
    raise WrongAnswerError('wrong wrong wrong!')
except WrongAnswerError:
    print('good good good')
finally:
    print('bye!')

good good good
bye!


In [60]:
class A:
    pass

raise A

TypeError: exceptions must derive from BaseException

In [61]:
BaseException.__bases__

(object,)

In [62]:
Exception.__bases__

(BaseException,)

In [63]:
TypeError.__bases__

(Exception,)

In [64]:
KeyboardInterrupt.__bases__

(BaseException,)